In [1]:
import matplotlib.pyplot as plt
import numpy as np
import rospy
from pycram.datastructures.world import World
from pycram.llms.llm_api import (explore_simulation_environment, task_strategy_planner, explore_designators, 
                                 act_context, obj_context, loc_context)
from pycram.worlds.bullet_world import BulletWorld, Object
from pycram.datastructures.pose import Pose
from pycram.datastructures.enums import ObjectType, WorldMode
from rospy import sleep
from openai import OpenAI
import json

openai_api_key = ""
# OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_API_KEY = openai_api_key
client = OpenAI(api_key=OPENAI_API_KEY)

Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Failed to import Giskard messages, the real robot will not be available
Could not import RoboKudo messages, RoboKudo interface could not be initialized
pybullet build time: Nov 28 2023 23:51:11


In [2]:
world = BulletWorld(mode=WorldMode.GUI)
pr2 = Object("pr2", ObjectType.ROBOT, "pr2.urdf")
# kitchen = Object("kitchen", ObjectType.ENVIRONMENT, "kitchen.urdf")
milk = Object("milk", ObjectType.MILK, "milk.stl", pose=Pose([1.3, 1, 0.9]))
cereal = Object("froot_loops", ObjectType.BREAKFAST_CEREAL, "breakfast_cereal.stl", pose=Pose([1.3, 0.9, 0.95]))
spoon = Object("spoon", ObjectType.SPOON, "spoon.stl", pose=Pose([1.3, 1.1, 0.87]))

Unknown tag "rgba_color" in /robot[@name='milk_object']/link[@name='milk_main']/visual[1]/material[@name='white']
Unknown tag "rgba_color" in /robot[@name='cereal_object']/link[@name='cereal_main']/visual[1]/material[@name='white']
Unknown tag "rgba_color" in /robot[@name='spoon_object']/link[@name='spoon_main']/visual[1]/material[@name='white']
Unknown tag "rgba_color" in /robot[@name='milk_object']/link[@name='milk_main']/visual[1]/material[@name='white']
Unknown tag "rgba_color" in /robot[@name='cereal_object']/link[@name='cereal_main']/visual[1]/material[@name='white']


spoon problem


Unknown tag "rgba_color" in /robot[@name='spoon_object']/link[@name='spoon_main']/visual[1]/material[@name='white']


In [59]:
env = explore_simulation_environment(world=world, prompt = "name all objects which are holdable in the world")
print(env)

```json
{
  "holdable_objects": [
    {
      "id": 5,
      "name": "spoon",
      "type": "SPOON"
    },
    {
      "id": 3,
      "name": "milk",
      "type": "MILK"
    },
    {
      "id": 4,
      "name": "froot_loops",
      "type": "BREAKFAST_CEREAL"
    }
  ]
}
```


In [8]:
task_res = task_strategy_planner(world=world, prompt="move robot")
print(task_res)


None


In [63]:
desig_res = explore_designators(world=world, prompt="place the milk on the table")
print(desig_res)


```python
# Assuming the necessary classes and methods are already defined and imported

# Define the object designator for the milk
milk_designator = ObjectDesignatorDescription(names=["milk"], types=[ObjectType.MILK])

# Define the target location for the table (assuming we have a way to get the table's pose)
table_pose = Pose(position=Position(x=1.5, y=1.0, z=0.95), orientation=Orientation(x=0.0, y=0.0, z=0.0, w=1.0))

# Define the action designator to place the milk on the table
place_action = PlaceAction(
    object_designator_description=milk_designator,
    target_locations=[table_pose],
    arms=["right_arm"]  # Assuming the robot has a right arm
)

# Ground the action to create a performable action
performable_action = place_action.ground()

# Perform the action
performable_action.perform()
```


In [65]:
from pydantic import BaseModel
from typing import List, Optional
from enum import Enum, auto
class ObjectType(Enum):
    MILK = auto()
    SPOON = auto()
    BOWL = auto()
    ROBOT = auto()
    ENVIRONMENT = auto()

class odd(BaseModel):
    name: str  # Name of the Object
    obj_type: str # Type of the Object
    
class ObjectDesignatorDescription(BaseModel):
    ObjectDesignatorDescription : List[odd]
    
class Environment(BaseModel):
    objects: List[str]  # Objects in the environment
    containers: List[str] # Objects Suitable for Holding Something in it
    holdables: List[str] # Objects that can be held
    placeables: List[str] # Places where objects can be placed

In [11]:
%%time
system = "You are an bullet world simulation environment analyst"
user = f"""Help me to identify the object designators with their names and types of all the objects
"""
assistant = f"Information of all objects in the world {str(world.objects)}"

completion = client.beta.chat.completions.parse(    
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": system},
    {"role": "assistant", "content": assistant},
    {"role": "user", "content": user},
  ],
  response_format=ObjectDesignatorDescription,
)

CPU times: user 2.9 s, sys: 11.9 ms, total: 2.91 s
Wall time: 2.68 s


In [13]:
print(completion.choices[0].message.content)

{
  "ObjectDesignatorDescription": [
    {
      "name": "floor",
      "obj_type": "ENVIRONMENT"
    },
    {
      "name": "pr2",
      "obj_type": "ROBOT"
    },
    {
      "name": "milk",
      "obj_type": "MILK"
    },
    {
      "name": "froot_loops",
      "obj_type": "BREAKFAST_CEREAL"
    },
    {
      "name": "spoon",
      "obj_type": "SPOON"
    }
  ]
}


In [53]:
from pycram.llms.llm_api import explore_simulation_environment, task_strategy_planner, explore_designators, act_context
context = str(world.objects)
context = context + f"\n\n Available Action Designators are: \n\n {act_context}"
promptin = "pick up the milk from the table and place it on the shelf"
response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system",
             "content": "resolve the task prompted by the user into a set of action designators and parameters needed to execute it, no additional text"},
            {"role": "assistant", "content": str(context)},
            {"role": "user", "content": promptin},
        ],
        max_tokens=1000,
        temperature=0,
    )

In [54]:
print(response.choices[0].message.content)

1. **PickUpAction**
   - object_designator_description: milk
   - arms: [left, right]
   - grasps: [left, right]

2. **PlaceAction**
   - object_designator_description: milk
   - target_locations: [shelf_pose]  # Define the pose of the shelf
   - arms: [left, right]


In [55]:
kitchen = Object("kitchen", ObjectType.ENVIRONMENT, "kitchen.urdf")

[ERROR] [1725965640.345255]: An object with the name kitchen already exists in the world.


In [51]:
kitchen.get_base_origin()

header: 
  seq: 0
  stamp: 
    secs: 1725965554
    nsecs: 260509014
  frame_id: "map"
pose: 
  position: 
    x: 0.0
    y: 0.0
    z: -0.001
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.0
    w: 1.0